In [1]:
# notebooks/helm_experiments.ipynb
import IPython
print(IPython.__version__)  # Check your IPython version

%load_ext autoreload
%autoreload 2
from dataclasses import dataclass
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoConfig
import json
from typing import List, Optional, Tuple, Dict
import time
from dataclasses import dataclass
import lm_eval
from lm_eval import evaluator, tasks, utils
from lm_eval.api.model import LM
from lm_eval.api.registry import register_model
from lm_eval.tasks import get_task_dict
from functools import partial
from tqdm import tqdm
import numpy as np
import multiprocessing
import ftfy
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from itertools import zip_longest

print(f"torch version: {torch.__version__}")
print(f"transformers version: {transformers.__version__}")

8.18.1


/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <4C793A59-B32A-3AF1-BEA5-03AD7C5C80C6> /opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/deep-learning-env/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/deep-learning-env/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/deep-learning-env/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there 

torch version: 2.6.0.dev20241112
transformers version: 4.46.3


In [2]:
import sys, os
sys.path.append('../')
from data.config import BenchmarkConfig, DatasetConfig
from data.data import DatasetManager, ModelDataset
from data.metrics import BenchmarkMetrics
from models.h2o.h2o_gptneox import GPTNeoXAttention_Mask, convert_kvcache_gpt_neox_heavy_recent
from models.h2o.h2o_llama import LlamaAttention_heavy_hitter, convert_kvcache_llama_heavy_recent
from models.h2o.h2o_opt import OPTAttention_Mask, convert_kvcache_opt_heavy_recent
from models.base_models import ModelLoader
from tests.run_benchmark import run_benchmark, run_helm_benchmark
from scripts.run_helm import run_experiment
import logging
logging.basicConfig(level=logging.INFO)


In [3]:
@dataclass
class Args:
    input_path: str
    output_path: str
    model_name: str
    model_arch: str
    cache_dir: str = "./cache"
    heavy_ratio: float = 0.1
    recent_ratio: float = 0.1
    enable_small_cache: bool = True
    sample_num: int = 10  # Start with a small sample size



In [9]:
# Test GPT-NeoX
args = Args(
    input_path="data/xsum/xsum.jsonl",
    output_path="results/xsum-gptneox-test.jsonl",
    model_name="EleutherAI/gpt-neox-20b",
    model_arch="gpt_neox"
)


In [4]:
args = Args(
    input_path="data/xsum/xsum.jsonl",
    output_path="results/xsum-llama2-test.jsonl",
    model_name="meta-llama/Llama-2-7b-hf",
    model_arch="llama"
)

In [6]:
run_experiment(args)

2024-12-03:09:25:39,962 INFO     [run_helm.py:17] Using device: mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-12-03:09:25:40,879 INFO     [run_helm.py:41] Enabling H2O Cache


FileNotFoundError: [Errno 2] No such file or directory: 'data/xsum/xsum.jsonl'